<a href="https://colab.research.google.com/github/GauravSahani1417/PySpark-Basic-Implementation/blob/main/Summarizing_a_DataFrame_in_PySpark_%7C_min%2C_max%2C_count%2C_percentile%2C_schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setting up the PySpark environment

# Install java 8
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Apache Spark binary: This link can change based on the version. Update this link with the latest version before using
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

# Unzip file
!tar -xf spark-3.0.2-bin-hadoop2.7.tgz

# Install findspark: Adds Pyspark to sys.path at runtime
!pip install -q findspark

# Install pyspark
!pip install pyspark

# Add environmental variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

# findspark will locate spark in the system
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Hands-on PySpark on Google Colab") \
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00603/in-vehicle-coupon-recommendation.csv -P sample_data/

In [ ]:
# We can set header='true' and inferSchema='true' to infer the schema while reading the data

filepath = "sample_data/in-vehicle-coupon-recommendation.csv"
spark_df = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+-------------+---+
|destination    |passanger|weather|temperature|time|coupon               |expiration|gender|age|maritalStatus    |has_children|education               |occupation|income         |car |Bar  |CoffeeHouse|CarryAway|RestaurantLessThan20|Restaurant20To50|toCoupon_GEQ5min|toCoupon_GEQ15min|toCoupon_GEQ25min|direction_same|direction_opp|Y  |
+---------------+---------+-------+-----------+----+---------------------+----------+------+---+-----------------+------------+------------------------+----------+---------------+----+-----+-----------+---------+--------------------+----------------+----------------+-----------------+-----------------+--------------+--------

In [ ]:
len(spark_df.columns)

26

In [ ]:
#Selecting 9/26 columns
columns_to_use = ["destination", "passanger", "weather", "temperature", "time", "coupon", "gender", "age", "has_children", "toCoupon_GEQ5min", "Y"]
spark_df = spark_df.select(*columns_to_use)
spark_df.show(5, truncate=False)

+---------------+---------+-------+-----------+----+---------------------+------+---+------------+----------------+---+
|destination    |passanger|weather|temperature|time|coupon               |gender|age|has_children|toCoupon_GEQ5min|Y  |
+---------------+---------+-------+-----------+----+---------------------+------+---+------------+----------------+---+
|No Urgent Place|Alone    |Sunny  |55         |2PM |Restaurant(<20)      |Female|21 |1           |1               |1  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Coffee House         |Female|21 |1           |1               |0  |
|No Urgent Place|Friend(s)|Sunny  |80         |10AM|Carry out & Take away|Female|21 |1           |1               |1  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |1               |0  |
|No Urgent Place|Friend(s)|Sunny  |80         |2PM |Coffee House         |Female|21 |1           |1               |0  |
+---------------+---------+-------+-----

In [ ]:
spark_df.count()

12684

In [ ]:
print(spark_df.columns)

['destination', 'passanger', 'weather', 'temperature', 'time', 'coupon', 'gender', 'age', 'has_children', 'toCoupon_GEQ5min', 'Y']


In [ ]:
len(spark_df.columns)

11

In [ ]:
spark_df.printSchema()

root
 |-- destination: string (nullable = true)
 |-- passanger: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- coupon: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- has_children: integer (nullable = true)
 |-- toCoupon_GEQ5min: integer (nullable = true)
 |-- Y: integer (nullable = true)



In [ ]:
spark_df.dtypes

[('destination', 'string'),
 ('passanger', 'string'),
 ('weather', 'string'),
 ('temperature', 'int'),
 ('time', 'string'),
 ('coupon', 'string'),
 ('gender', 'string'),
 ('age', 'string'),
 ('has_children', 'int'),
 ('toCoupon_GEQ5min', 'int'),
 ('Y', 'int')]

Describe the dataframe (min, max, count)

In [ ]:
spark_df.describe()

DataFrame[summary: string, destination: string, passanger: string, weather: string, temperature: string, time: string, coupon: string, gender: string, age: string, has_children: string, toCoupon_GEQ5min: string, Y: string]

In [ ]:
# To get the output, you have to run action commands (like show, collect, etc.)
spark_df.describe().show()

+-------+-----------+---------+-------+------------------+-----+---------------+------+------------------+------------------+----------------+------------------+
|summary|destination|passanger|weather|       temperature| time|         coupon|gender|               age|      has_children|toCoupon_GEQ5min|                 Y|
+-------+-----------+---------+-------+------------------+-----+---------------+------+------------------+------------------+----------------+------------------+
|  count|      12684|    12684|  12684|             12684|12684|          12684| 12684|             12684|             12684|           12684|             12684|
|   mean|       null|     null|   null|63.301797540208135| null|           null|  null|29.887815247850035|0.4141438032166509|             1.0|0.5684326710816777|
| stddev|       null|     null|   null| 19.15448575684057| null|           null|  null| 7.697275065801651|   0.4925929797555|             0.0| 0.495314356461186|
|    min|       Home|    Alo

Percentiles

In [ ]:
spark_df.summary().show()

+-------+-----------+---------+-------+------------------+-----+---------------+------+------------------+------------------+----------------+------------------+
|summary|destination|passanger|weather|       temperature| time|         coupon|gender|               age|      has_children|toCoupon_GEQ5min|                 Y|
+-------+-----------+---------+-------+------------------+-----+---------------+------+------------------+------------------+----------------+------------------+
|  count|      12684|    12684|  12684|             12684|12684|          12684| 12684|             12684|             12684|           12684|             12684|
|   mean|       null|     null|   null|63.301797540208135| null|           null|  null|29.887815247850035|0.4141438032166509|             1.0|0.5684326710816777|
| stddev|       null|     null|   null| 19.15448575684057| null|           null|  null| 7.697275065801651|   0.4925929797555|             0.0| 0.495314356461186|
|    min|       Home|    Alo

In [ ]:
#Custom percentile summary!
spark_df.summary("40%", "60%", "90%").show()

+-------+-----------+---------+-------+-----------+----+------+------+----+------------+----------------+---+
|summary|destination|passanger|weather|temperature|time|coupon|gender| age|has_children|toCoupon_GEQ5min|  Y|
+-------+-----------+---------+-------+-----------+----+------+------+----+------------+----------------+---+
|    40%|       null|     null|   null|         55|null|  null|  null|26.0|           0|               1|  0|
|    60%|       null|     null|   null|         80|null|  null|  null|31.0|           1|               1|  1|
|    90%|       null|     null|   null|         80|null|  null|  null|41.0|           1|               1|  1|
+-------+-----------+---------+-------+-----------+----+------+------+----+------------+----------------+---+



Hence, We have seen how to extract basic statistics from a DataFrame using PySpark